In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:
import qiskit
qiskit.version.get_version_info()

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.circuit.library import QFT,PhaseEstimation

import numpy as np
from numpy.random import randint
from math import gcd,lcm
import matplotlib.pyplot as plt

In [ ]:
N = 15
a = 7

xvals = range(N)
yvals = [1]
y = 1
for x in range(1,N):
    y = np.mod(a*y, N)
    yvals += [y] 

fig, ax = plt.subplots()
ax.plot(xvals, yvals, linewidth=1, linestyle='solid', marker='o')
ax.set(xlabel='$x$', ylabel=f'${a}^x$ mod ${N}$')

try: 
    r = yvals[1:].index(1) + 1
    plt.annotate('', xy=(0,1), xytext=(r,1),
                 arrowprops=dict(arrowstyle='<->'))
    plt.annotate(f'$r={r}$', xy=(r/3,1.5))
except ValueError:
    print(f'An order not found, {a} may be larger than {N} or {a} and {N} has a common factor {gcd(a,N)}.')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,1), dpi=100)
ax = fig.add_subplot(111)
plt.subplots_adjust()
ax.spines["top"].set_color("None")
ax.spines["right"].set_color("None")
#ax.spines["bottom"].set_color("None")
ax.spines["left"].set_color("None")
ax.set_yticks([],labels="")
N = 7
x = {0.0:'0', 1.0:'1'}
for denom in range(N,-1,-1):
    for num in range(1,denom):
        x[float(num)/denom] = f'{num}/{denom}'
x_sorted = sorted(x.items(), key=lambda x:x[0])
x_ticks = [x[0] for x in x_sorted]
x_labels = [x[1] for x in x_sorted]
print(x_ticks)
ax.set_xticks(x_ticks,labels=x_labels)
print(x_sorted)
mark = float(108)/128
ax.annotate("108/128",(mark,0),(mark-0.025,0.7), arrowprops={"arrowstyle":"->", "color":"0.0"})

plt.show()


In [ ]:
from fractions import Fraction
# 분모가 7 이하인 유리수 중에서 108/128 과 가장 가까운 유리수의 분자와 분모를 찾는다.
f = Fraction(108/128).limit_denominator(7)
r = f.denominator
s = f.numerator
print(f"estimated phase = {s}/{r}")

In [ ]:
import numpy as np
from numpy.random import randint
from math import gcd,lcm
# factoring N=21
N = 21  # number to be factored
n =  5   # number of qubits representing N
t =  2*n + 1 # number of evaluation qubits
a =  randint(2,N)
g = gcd(a,N)
print(f"a = {a}, gcd({a},{N}) = {g}") # if g != 1, g is a factor of N

In [ ]:
# generate U matrix of (a*i) mod N
two_power_n = 1 << n # 2^n
U = np.eye(two_power_n) # Identity matrix
for j in range(N):
    U[j][j] = 0
    U[(a*j)%N][j] = 1
# U 행렬을 구현하는 양자 회로 qcU 를 생성
qcU = QuantumCircuit(n)
qcU.unitary(U, range(n))


In [ ]:

from qiskit.visualization import array_to_latex
array_to_latex(U, prefix="U = ", max_size=32)

In [ ]:
# phase estimation using QPE
from qiskit.circuit.library import PhaseEstimation

# generate a order-finding circuit
qx = QuantumRegister(t,'x')
qy = QuantumRegister(n,'y')
c = ClassicalRegister(t,'c')
qc_of = QuantumCircuit(qx, qy, c)
qc_of.x(qy[0])                 # input state = |1>
qc_of.append(PhaseEstimation(t,qcU),qx[:]+qy[:])
qc_of.measure(qx,reversed(c))
qc_of.draw(output='mpl',fold=-1)

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_circuit = pm.run(qc_of)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.c.get_counts()
print(count)

In [ ]:
phase = list(count.keys())
print("measured output state = ", phase[0])

In [ ]:
from fractions import Fraction
s_pre = int(phase[0],2) # convert a binary string to an integer
r_pre = 1<<t  # r_pre == 2^t
print(f"measured phase = {s_pre}/{r_pre}")
f = Fraction(s_pre/r_pre).limit_denominator(N)
r = f.denominator
s = f.numerator
print(f"estimated phase = {s}/{r}")
print(f"estimated order r = {r}")

In [ ]:
def estimate_order(sampler, isa_circuit, t):
    job = sampler.run([isa_circuit], shots=1)
    result = job.result()
    count = result[0].data.c.get_counts()
    phase = list(count.keys())
    s_pre = int(phase[0],2) # convert a binary string to an integer
    r_pre = 1<<t  # r_pre == 2^t
    f = Fraction(s_pre/r_pre).limit_denominator(N)
    return  f.denominator

def confirm_order(a, r, N):
    return (a**r % N == 1)

if confirm_order(a, r, N):
    found = True
else : 
    r1 = r
    r2 = estimate_order(sampler, isa_circuit, t)
    if confirm_order(a, r2, N):
        r = r2
        found = True
    else :
        r = lcm(r1, r2)
        if r >= N :
            found = False
        elif confirm_order(a, r, N):
            found = True
        else :
            found = False
    print(f"r1={r1} r2={r2} estimated r={r}")

if (found) :
    print(f"estimated value {r} is an order of {a} for {N}")
else :
    print(f"{r} is NOT an order of {a} for {N} because {a}^{r} % {N} = {a**r%N}")

In [ ]:
if r % 2 == 0 : # r is even
    a_power_r_over2 = a**int(r/2)
    p = gcd(a_power_r_over2+1,N)
    print (f"p = gcd( {a}^({r}/2) + 1 (= {a_power_r_over2+1}), {N} ) = {p}")
    if (p == N)  :
        print (f"{a}^({r}/2) + 1 (= {a_power_r_over2+1}) is a multiple of {N}.")
        print (f" a = {a} is not lucky. try again.")
    elif (p == 1) :
        print (f" r = {r} is not an order of {a}. try again.")
    else :
        q = gcd(a_power_r_over2-1,N)
        print (f"q = gcd( {a}^({r}/2) - 1 (= {a_power_r_over2-1}), {N} ) = {q}")
        print (f"{p} and {q} are factors of {N}")
else :
    print (f"r = {r} is an odd number. a = {a} is not lucky. try again.")